In [1]:
import os
import pickle
import binascii

def crc32_checksum(filename):
    buf = open(filename,'rb').read()
    buf = (binascii.crc32(buf) & 0xFFFFFFFF)
    return "%08X" % buf

def get_kcs(path_192) :
    crc32 = str(crc32_checksum(path_192))
    size = str(os.path.getsize(path_192))
    f = os.path.split(path_192)[1]
    key = f[:8]
    kcs = key + crc32 + size 
    return kcs

def read_cks(path_kcs) :
    with open(path_kcs, "rb") as pkl :
        return pickle.load(pkl)

def delete_kcs(filePath, dict_kcs) :
    k = os.path.split(filePath)[1][:8]
    c = str(crc32_checksum(filePath))
    s = str(os.path.getsize(filePath))
    kcs = k + c + s
    if kcs in dict_kcs :
        print(f"kcs : {kcs}   path : {dict_kcs[kcs]}")
        del dict_kcs[kcs]
        print("삭제하였습니다.")
    else :
      print(filePath, kcs, "없는 키값")

def write_cks(path_kcs, dict_kcs) :
    with open(path_kcs, "wb") as pkl :
        pickle.dump(dict_kcs, pkl)

path_kcs = r"\\192.168.0.75\스캔파일\스캔파일log\_project\파일\중복조사\kcs별 파일정보_솔림헬프.pkl"

In [5]:
# 읽기
dict_kcs = read_cks(path_kcs)
print("최초dict_kcs개수 :", len(dict_kcs))

최초dict_kcs개수 : 325432


In [6]:
# 삭제하기
####################################
path_192 = r"\\192.168.0.75\솔림헬프\5.등초본\한울가람\20422521\20422521_김형권_운전면허증.pdf"
####################################
delete_kcs(path_192, dict_kcs)

kcs : 204225211AD290C2286152   path : /volume1/솔림헬프/5.등초본/한울가람/20422521/20422521_김형권_초본_221012.pdf
삭제하였습니다.


In [3]:
# 폴더 내 파일 전체 삭제
path = r"D:\업로드완료\서울 채권별 20우담 1매입전 양통"
for f in os.listdir(path) :
    path_192 = os.path.join(path, f)
    delete_kcs(path_192, dict_kcs)


kcs : 204601092D37AAF7188996   path : /volume1/솔림헬프/1.원인서류/우담-01/20460109/20460109_박성재_원인서류_양통 우담.pdf
삭제하였습니다.
kcs : 20460109686764B651372   path : /volume1/솔림헬프/1.원인서류/우담-01/20460109/20460109_박성재_원인서류_양통 우담.pdf
삭제하였습니다.
kcs : 204602105DEC07C062656   path : /volume1/솔림헬프/1.원인서류/우담-01/20460210/20460210_김나현_원인서류_양통 우담.pdf
삭제하였습니다.
kcs : 204603144558CEF566899   path : /volume1/솔림헬프/1.원인서류/우담-01/20460314/20460314_김성락_원인서류_양통 우담.pdf
삭제하였습니다.
kcs : 204604548746A61D81719   path : /volume1/솔림헬프/1.원인서류/우담-01/20460454/20460454_한광복_원인서류_양통 우담.pdf
삭제하였습니다.
kcs : 20461393B064D53081435   path : /volume1/솔림헬프/1.원인서류/우담-01/20461393/20461393_박분희_원인서류_양통 우담.pdf
삭제하였습니다.
kcs : 2046142700CF06CA76559   path : /volume1/솔림헬프/1.원인서류/우담-01/20461427/20461427_김지연_원인서류_양통 우담.pdf
삭제하였습니다.
kcs : 20462009EB4B917692243   path : /volume1/솔림헬프/1.원인서류/우담-01/20462009/20462009_김철우_원인서류_양통 우담.pdf
삭제하였습니다.
kcs : 204620248D4902A2134829   path : /volume1/솔림헬프/1.원인서류/우담-01/20462024/20462024_김병환_원인서류_양통 우담.pdf
삭제하였습니다.
kcs : 20

In [7]:
# 쓰기
write_cks(path_kcs, dict_kcs)